# Лабораторная работа №1

In [135]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет

In [136]:
mushrooom_dataset = pd.read_csv('MushroomDataset/secondary_data.csv',delimiter=';')

Выделяем таргетный столбец в отдельный Series, а все остальные столбы в DataFrame. Также удаляем все столбцы, где большая часть данных типа NaN и заменяем все классы на номер буквы в алфавите.

In [144]:
y = mushrooom_dataset['class']
X = mushrooom_dataset.drop('class',axis=1)
X = X.drop(columns=['cap-surface','gill-spacing','gill-attachment','stem-root',
                    'stem-surface','veil-type','veil-color','ring-type','spore-print-color'])
X.replace({'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,
           'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,
           'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,
           't':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26},inplace=True)

После устранения пропусков нормализуем все значения.

In [146]:
scaler = StandardScaler()
X_ans = scaler.fit_transform(X)

Разделяем данные на тренировочную и тестовую части.

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.2)

Объявим модель, на основе которой будет проходить классификация. В качестве начальных параметров зададим: кол-во соседей = 10, весомость соседа на основе растояния до него, размер листьев, который используется при построении деревьев решений = 30, расчёт расстояния по евклидовой метрике, и кол-во паралельных потоков = 4.

In [148]:
best_model = KNeighborsClassifier(
   n_neighbors=10, 
   weights='distance',
   algorithm='auto',
   leaf_size=30,
   metric='euclidean',
   metric_params=None,
   n_jobs=4
)

Далее натренируем модель, для этого передадит в неё тренировочные данные, и проведём тренировку для кол-ва соседей от 1 до 30 после чего запомним самые лучшие параметры.

In [149]:
best_model.fit(X_train, y_train)
model_params = best_model.get_params()
tuned_params = {}
for k, v in model_params.items():
    tuned_params[k] = [v]
tuned_params['n_neighbors'] = range(1, 30)
tuned_params['metric'] = ['minkowski','euclidean','manhattan']
clf = GridSearchCV(best_model, tuned_params, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)
best_params = clf.best_params_

После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [150]:
best_model = KNeighborsClassifier(**best_params)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print('Used params:',best_params)
print('Evaluation:\n', metrics.classification_report(y_test, predicted,digits=5))
print('Score:',clf.score(X_test,y_test))

Used params: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'manhattan', 'metric_params': None, 'n_jobs': 4, 'n_neighbors': 4, 'p': 2, 'weights': 'distance'}
Evaluation:
               precision    recall  f1-score   support

           e    0.99591   0.99461   0.99526      5383
           p    0.99576   0.99678   0.99627      6831

    accuracy                        0.99582     12214
   macro avg    0.99583   0.99570   0.99576     12214
weighted avg    0.99582   0.99582   0.99582     12214

Score: 0.9958244637301458
